In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommenders: Movielens100k, load_dataset, ratio_split, BPR, evaluate_u2i, MeanPrecision, MeanRecall, MeanNDCG, fit!, predict_u2i, make_u2i_dataset

In [2]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [3]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))

(64000, 16000, 20000)

In [4]:
prec10 = MeanPrecision(10)
recall10 = MeanRecall(10)
ndcg10 = MeanNDCG(10)
metrics = [prec10, recall10, ndcg10]

3-element Vector{Recommenders.MeanMetric}:
 Recommenders.MeanMetric{Recommenders.Precision}(Recommenders.Precision(10, "precision"))
 Recommenders.MeanMetric{Recommenders.Recall}(Recommenders.Recall(10, "recall"))
 Recommenders.MeanMetric{Recommenders.NDCG}(Recommenders.NDCG(10, "ndcg"))

In [5]:
model = BPR(2^9, 0.015587614364453028)

BPR(512, Recommenders.BPRLoss(), 0.015587614364453028, nothing, nothing, nothing, nothing, nothing, nothing)

In [6]:
evaluate_u2i(model, train_valid_table, test_table, metrics, 10, col_item=:movieid, n_epochs=128, learning_rate=0.007785000886303088, n_negatives=12, drop_history=true, verbose=16)

┌ Info: epoch=16: train_loss=0.15129748219506578, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=32: train_loss=0.14974151893965998, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=48: train_loss=0.1485660155304567, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=64: train_loss=0.14852505953421827, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=80: train_loss=0.14878208646946864, val_metric=0.0, best_val_metric=0.0, best_epoch=1
└ @ Recommenders /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/bpr.jl:133
┌ Info: epoch=96: train_loss=0.1492600311157595, val_metric=0.0, best_val_me

(ndcg10 = 0.4083240486460882, precision10 = 0.34634146341463407, recall10 = 0.23163689943564778)